## Wk3 Proj Submission Toronto Neighborhood Clustering
***************************

### Import the required libraries 

In [1]:
import requests # requests Library
import random # library for random number generation
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # Library to handle html

### Read Canada webpage data
Use the Request Library to read Wikipedia site data into a variable

In [3]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #create a link varible containing the html address
page = requests.get(link) # create variable that conatins the returned resulsts from request



Use the Beautiful Soup library to parce webpage text data.
Identitify the desired table and add table data to a Pandas DataFrame. 
<br/>
View DataFrame sample 

In [4]:
page_soup = BeautifulSoup(page.text, 'html.parser') # use Beautiful Soup html parser to handle html code
t = page_soup.find('tbody') # Find First Table find_all tables also an option (Postal codes are included in the first table)
col = t.find_all('th') # Identify the Col Names to create empty Data Frame
rows = t.find_all('tr')[1:] # Identify Row Data to insert into empty Data Frame 

# loop through Html Columns, convert to strings and store in list t1
t1 = []
for i in range(len(col)):
    t1.append(str(col[i]).strip('<th>').strip('</').strip('\n'))

hoods = pd.DataFrame(columns = t1) # create data frame with colums headers using t1 list 

# loop through table rows contained in rows variable and add to the dataframe 
for i in range(len(rows)):
    hoods.loc[i, 'Postcode'] = rows[i].findAll(text=True)[1]
    hoods.loc[i, 'Borough'] = rows[i].findAll(text=True)[3]
    hoods.loc[i, 'Neighbourhood'] = str(rows[i].findAll(text=True)[5]).strip('\n')

print("DataFrame Shape:", hoods.shape, "Rows:", hoods.shape[0], "Columns:", hoods.shape[1]) # inspect dataframe shape
hoods.head() # review dataframe sample

DataFrame Shape: (289, 3) Rows: 289 Columns: 3


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Clean and prepare data for clustering 

Drop all rows where Borough is "Not assigned"

In [8]:
df1 = hoods.drop(hoods[hoods.Borough == 'Not assigned'].index) # drop all row's where Borough is equal to "Not assigned" and create a new dataframe
df1 = df1.reset_index(drop=True) # reset new dataframe index  
print("DataFrame Shape:", df1.shape, "Rows:", df1.shape[0], "Columns:", df1.shape[1] ) # inspect new dataframe shape (it should contain less rows)
df1.head() # display a 10 row sample of the new dataframe (Showing no Boroughs = "Not assigned")

DataFrame Shape: (212, 3) Rows: 212 Columns: 3


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Update all rows where Neighborhood is "Not assigned". Replace "Not assigned" with the Borough name.

In [10]:
# Iterate over the Values in the "Neighborhood" column and update None or Not Assigned values with the Borough name
for i in range(len(df1)):
    if df1['Neighbourhood'][i] == "Not assigned":
        df1['Neighbourhood'][i] = df1['Borough'][i]
    else:
        pass
print("DataFrame Shape:", df1.shape, "Rows:", df1.shape[0], "Columns:", df1.shape[1] ) # inspect dataframe shape (it should not change (212,3))
df1.head() # display a sample of the updated dataframe (all Niehborhoods now showing a value)

DataFrame Shape: (212, 3) Rows: 212 Columns: 3


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Group data by Post Code.<br/>
And Include each Unique Listed Neighborhood 

In [12]:
boro = {k: g["Borough"].tolist() for k,g in df1.groupby("Postcode")} # Create dictionary with Postal code as the Key, and all corresponding Borough's as a list
hood = {k: g["Neighbourhood"].tolist() for k,g in df1.groupby("Postcode")} # Create a dictionary with Postal code as the Key, and all corresponding Neiborhood's as a list
t2 = pd.DataFrame(list(boro.keys()), columns = ['PostCode'])
for i, row in t2.iterrows():
    t2.loc[i,'Borough'] = list(set(list(boro.values())[i]))[0]    
    t2.loc[i,'Neighbourhood'] = ', '.join(list(set(list(hood.values())[i])))

print("DataFrame Shape:", t2.shape, "Rows:", t2.shape[0], "Columns:", t2.shape[1] ) # inspect dataframe shape (it should not change)
t2.head(10)

DataFrame Shape: (103, 3) Rows: 103 Columns: 3


,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


Check table results for accuracy

In [13]:
t2[(t2['PostCode'] == 'M5A')] # test results by verifying 'M5A' Borough and Neighbourhood

,PostCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


### Adding Longitude and Latitude
--------

Use Requests Library to get Longitude and Latitude information for Postcodes

In [15]:
url="http://cocl.us/Geospatial_data" # create url variable
HoodCords=pd.read_csv(url) # read longitude latidude csv file information with url  
HoodCords.rename(index=str,columns={'Postal Code':'PostCode'},inplace=True) # update Postal Code Column name for dataframe merge 

Merge PostCode DataFrame with the Longitude and Latitude DataFrame 

In [16]:
Hoods_LongLaT = pd.merge(t2, HoodCords, how='left', on='PostCode') # Merge Toronto Neighborhood dataframe (t2) with Long Lat dataframe on Like Postal Codes 
print("DataFrame Shape:", Hoods_LongLaT.shape, "Rows:", Hoods_LongLaT.shape[0], "Columns:", Hoods_LongLaT.shape[1] ) # inspect dataframe shape with 2 new columns (row count should not change)
Hoods_LongLaT.head(15)

DataFrame Shape: (103, 5) Rows: 103 Columns: 5


,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


The DataFrame contains 103 rows of information for 11 Unique Boroughs.<br/>
Each borough may contain several different neighborhods, associated to several postal codes.  

In [23]:
print("Unique Borough Count:", len(Hoods_LongLaT['Borough'].unique()))
print("Unique Borough List", Hoods_LongLaT['Borough'].unique())

Unique Borough Count: 11
Unique Borough List ['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


### Neighborhood Analysis and Clustering
***********

Import Plottnig and Mapping Libraries

In [25]:
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans # import k-means from clustering stage
from sklearn.datasets.samples_generator import make_blobs

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import json # library to handle JSON files
import folium # folium for map rendering

Display a sample map of Toronto with all PostCode marked and labeled 

In [26]:
lat = 43.70011
long =-79.4163
Toronto_Map = folium.Map(location=[lat, long], zoom_start=11)
# add markers to map
for lat, lng, label, hood in zip(Hoods_LongLaT['Latitude'], Hoods_LongLaT['Longitude'], Hoods_LongLaT['PostCode'], Hoods_LongLaT['Neighbourhood']):
    label = folium.Popup("PostCode:"+label+"\n"+"Neighborhood(s):"+hood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_Map) 

In [27]:
Toronto_Map

Double-click __here__ FourSquare Authentication.
<!--
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190101' # Foursquare API version
-->

Assign Data Return limits and a radius for Four Square

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Define a function that will make each FourSquare API call in relation to the given PostCode and provided Log. & Lat. values. <br/>
This function will print the PostCode and Borough name for each API call.<br/>
It will place the results in a new DataFrame

In [40]:
def getNearbyVenues(names, hoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, hood, lat, lng in zip(names, hoods, latitudes, longitudes):
        print(name, hood)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            hood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostCode', 'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use getNearbyVenues function to make FourSquare API calls and store the results.

In [41]:
TorontoVenues = getNearbyVenues(names=Hoods_LongLaT['PostCode'],
                                   hoods=Hoods_LongLaT['Borough'],
                                   latitudes=Hoods_LongLaT['Latitude'],
                                   longitudes=Hoods_LongLaT['Longitude']
                                  )

M1B Scarborough
M1C Scarborough
M1E Scarborough
M1G Scarborough
M1H Scarborough
M1J Scarborough
M1K Scarborough
M1L Scarborough
M1M Scarborough
M1N Scarborough
M1P Scarborough
M1R Scarborough
M1S Scarborough
M1T Scarborough
M1V Scarborough
M1W Scarborough
M1X Scarborough
M2H North York
M2J North York
M2K North York
M2L North York
M2M North York
M2N North York
M2P North York
M2R North York
M3A North York
M3B North York
M3C North York
M3H North York
M3J North York
M3K North York
M3L North York
M3M North York
M3N North York
M4A North York
M4B East York
M4C East York
M4E East Toronto
M4G East York
M4H East York
M4J East York
M4K East Toronto
M4L East Toronto
M4M East Toronto
M4N Central Toronto
M4P Central Toronto
M4R Central Toronto
M4S Central Toronto
M4T Central Toronto
M4V Central Toronto
M4W Downtown Toronto
M4X Downtown Toronto
M4Y Downtown Toronto
M5A Downtown Toronto
M5B Downtown Toronto
M5C Downtown Toronto
M5E Downtown Toronto
M5G Downtown Toronto
M5H Downtown Toronto
M5J Downtow

Inspect the resulting DataFrame by reviewing the shape and a sample of the first 15 rows and the amount of unique Categories.


In [72]:
print('There are {} uniques categories.'.format(len(TorontoVenues['Venue Category'].unique())))
print(TorontoVenues.shape)
TorontoVenues.head(15)

There are 277 uniques categories.
(2233, 8)


,PostCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,Scarborough,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,M1E,Scarborough,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,M1E,Scarborough,43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,M1E,Scarborough,43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
8,M1G,Scarborough,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
9,M1G,Scarborough,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop


Use One Hot Encoding to create a numeric value based on each Category for each PostCode  <br/>

In [78]:
# one hot encoding
tv_onehot = pd.get_dummies(TorontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
tv_onehot['PostCode'] = TorontoVenues['PostCode'] 

# move postcode column to the first column
fixed_columns = [tv_onehot.columns[-1]] + list(tv_onehot.columns[:-1])
tv_onehot = tv_onehot[fixed_columns]

tv_onehot.head(5)

,PostCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the One Hot Encoded DataFrame by PostCode for Clustering

In [85]:
print('DataFrame Shape After One Hot encoding:', tv_onehot.shape)

tv_grp = tv_onehot.groupby('PostCode').mean().reset_index() # Group One Hot encoded DataFrame by PostCode

print("DataFrame Shape after grouping by PostCode:", tv_grp.shape)
tv_grp.head()

DataFrame Shape After one Hot encoding: (2233, 278)
DataFrame Shape after grouping by PostCode: (100, 278)


,PostCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Define a function that takes the one hot encoded values and identifies the 10 most common venues for each PostCode. <br/>
Visualizing this data will help to review the Common Venue rankings when comparing PostCodes that have been clustered together.

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostCode'] = tv_grp['PostCode']

for ind in np.arange(tv_grp.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tv_grp.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,PostCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Mexican Restaurant,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Rental Car Location,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store
3,M1G,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Lounge,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Yoga Studio,Doner Restaurant


Run the Kmeans Clustering alogrithim in an attempt to Cluster PostCodes <br/>
Print a sample of the Cluster Labels

In [89]:
# set number of clusters
kclusters = 5
torontoGrpClust = tv_grp.drop('PostCode', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoGrpClust) # run k-means clustering
kmeans.labels_[0:10] # check cluster labels generated for each row in the dataframe

array([0, 2, 2, 2, 2, 0, 2, 2, 2, 2], dtype=int32)

Insert the new Cluster Labels clumns into the sorted venues datatframe <br/>
Then merge sorted venues DataFrame with the Most Common Venues Data Frame 

In [90]:
# add clustering labels
pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tMerged = Hoods_LongLaT
tMerged = tMerged.join(pc_venues_sorted.set_index('PostCode'), on='PostCode') # merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood

tMerged.head() # check the last columns!

,PostCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711,2.0,Mexican Restaurant,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Rental Car Location,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Lounge,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Yoga Studio,Doner Restaurant


In [93]:
tMerged['Cluster Labels'].fillna(0, inplace=True)

Use Folium to plot a new Map of Toronto PostCodes where Point Color corresponds to Assigned Cluster Value 

In [95]:
# create map
Toronto_clusters = folium.Map(location=[lat, long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tMerged['Latitude'], tMerged['Longitude'], tMerged['PostCode'], tMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(round(cluster))-1],
        fill_opacity=0.7).add_to(Toronto_clusters)    
Toronto_clusters

### Inspecting Cluster results
Print each PostCode that was assigned to the selected cluster.
* Cluster 0: Has a High Frequency of Coffee Shops in the top results
* Cluster 1: Has 2 Post codes and The Most common venues for those are a Baseball field, and a yoga studio. These two track very closely in all top venues.
* Cluster 2: Have Parks, Yoga Studios and a dumpling Restaurant in the top venues 
* Cluster 3: Has the Fewest with only 1 PostCode where  the Bank, Eastern European Restaurants and Discount Stores are the top venues  
* Cluster 4: Has the most Post codes and seems to have the Highest frequency of Restaurants. As the most Common Venues

In [73]:
tMerged.loc[tMerged['Cluster Labels'] == 0, tMerged.columns[[0] + list(range(5, tMerged.shape[1]))]]

,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M1G,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
6,M1K,0.0,Department Store,Train Station,Discount Store,Coffee Shop,Chinese Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore
16,M1X,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,M2R,0.0,Pizza Place,Grocery Store,Coffee Shop,Butcher,Pharmacy,Greek Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
29,M3J,0.0,Bar,Massage Studio,Coffee Shop,Caribbean Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
34,M4A,0.0,Nail Salon,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run
37,M4E,0.0,Pub,Health Food Store,Coffee Shop,Neighborhood,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
48,M4T,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,M4V,0.0,Pub,Coffee Shop,Supermarket,Light Rail Station,Vietnamese Restaurant,Fried Chicken Joint,Pizza Place,Bagel Shop,Convenience Store,American Restaurant
53,M5A,0.0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Mexican Restaurant,Theater,Spa,Beer Store


In [74]:
tMerged.loc[tMerged['Cluster Labels'] == 1, tMerged.columns[[0] + list(range(5, tMerged.shape[1]))]]

,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,M8Y,1.0,Baseball Field,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
97,M9M,1.0,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Empanada Restaurant


In [75]:
tMerged.loc[tMerged['Cluster Labels'] == 2, tMerged.columns[[0] + list(range(5, tMerged.shape[1]))]]

,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M1V,2.0,Playground,Coffee Shop,Park,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
20,M2L,2.0,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
23,M2P,2.0,Bar,Bank,Park,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
30,M3K,2.0,Park,Airport,Bus Stop,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,M4J,2.0,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
50,M4W,2.0,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,M6E,2.0,Park,Pharmacy,Market,Fast Food Restaurant,Women's Store,Grocery Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
90,M8X,2.0,River,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
100,M9R,2.0,Pizza Place,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [76]:
tMerged.loc[tMerged['Cluster Labels'] == 3, tMerged.columns[[0] + list(range(5, tMerged.shape[1]))]]

,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,M9B,3.0,Bank,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Field


In [77]:
tMerged.loc[tMerged['Cluster Labels'] == 4, tMerged.columns[[0] + list(range(5, tMerged.shape[1]))]]

,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,4.0,Fast Food Restaurant,Print Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M1C,4.0,Bar,Golf Course,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
2,M1E,4.0,Breakfast Spot,Spa,Pizza Place,Electronics Store,Rental Car Location,Medical Center,Mexican Restaurant,Donut Shop,Discount Store,Dog Run
4,M1H,4.0,Hakka Restaurant,Bank,Lounge,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Bakery,Athletics & Sports,Dog Run,Doner Restaurant
5,M1J,4.0,Pizza Place,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,M1L,4.0,Bakery,Bus Line,Fast Food Restaurant,Intersection,Metro Station,Bus Station,Soccer Field,Park,Cosmetics Shop,Concert Hall
8,M1M,4.0,Motel,American Restaurant,Skating Rink,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
9,M1N,4.0,College Stadium,Café,General Entertainment,Skating Rink,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
10,M1P,4.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run
11,M1R,4.0,Auto Garage,Breakfast Spot,Sandwich Place,Shopping Mall,Bakery,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop


### After reviewing Cluster results edit Kmeans calculation and re-run to review result differences <br/>
Increased from 5 Clusters to 8 to see if the data would be grouped in a similar fashion
Plot results with Toronto Clusters 2 Map

Additional Results review and comparison can be done by displaying the PostCode by Cluster Group as done previsouly 


In [62]:
# set number of clusters
kclusters2 = 8

torontoGrpClust2 = tv_grp.drop('PostCode', 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters2, random_state=0).fit(torontoGrpClust2)

# check cluster labels generated for each row in the dataframe
kmeans2.labels_

array([1, 4, 2, 2, 2, 6, 2, 0, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 3, 2, 7,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2,
       2, 2, 6, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 7, 2, 2, 5, 2, 2, 0, 2, 2], dtype=int32)

In [63]:
# add clustering labels
pcs2.insert(0, 'Cluster Labels', kmeans2.labels_)

tMerged2 = Hoods_LongLaT

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tMerged2 = tMerged2.join(pcs2.set_index('PostCode'), on='PostCode')

tMerged2.head() # check the last columns!

,PostCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711,2.0,Mexican Restaurant,Pizza Place,Electronics Store,Breakfast Spot,Medical Center,Rental Car Location,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Lounge,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Yoga Studio,Doner Restaurant


In [64]:
tMerged2['Cluster Labels'].fillna(0, inplace=True)

In [65]:
# create map
Toronto_clusters2 = folium.Map(location=[lat, long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters2)
ys = [i + x + (i*x)**2 for i in range(kclusters2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tMerged2['Latitude'], tMerged2['Longitude'], tMerged2['PostCode'], tMerged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(Toronto_clusters2)
       
Toronto_clusters2